In [13]:
from dis import dis
from osgeo import gdal
import os, sys
import imageio
import numpy as np
import pandas as pd
# from PIL import Image

# load GDAL (Geospatial Data Abstraction Library) driver for tiff files
driver = gdal.GetDriverByName('GTiff')

In [14]:
tif_name = "./allanite_base.tif"

df = pd.read_csv('./Water Folder/Seasonal_for_tif.csv')

final_name = "MapFC.tif"

ds = gdal.Open(tif_name)
# ds = gdal.Open("Months/sentinel2a_Jamui_2021_01.tif")

# TIFF to CSV 
# first option
xyz = gdal.Translate("dem.xyz", ds)
xyz = None

df1 = pd.read_csv("dem.xyz", sep = " ", header = None)
df1.columns = ["x","y", "value"]
df1.drop('value', inplace=True, axis=1)

In [15]:
def distance(a,b):
    return (a[0] - b[0])**2 + (a[1] - b[1])**2

In [16]:


dall = df1.copy()


# df['x'] = df['geo']
# df_all = pd.read_csv('map.csv')
# df.drop('index', inplace=True, axis=1)

# dall = pd.merge(df1, df, how="left",left_on=['x','y'], right_on = ['x','y'])

# df['ax'] = round(df['x'],4)
# df1['ax'] = round(df1['x'],4)

# df['xy'] = list(zip(df.x, df.y))
# df1['xy'] = list(zip(df1.x, df1.y))

# dall = pd.merge_asof(df1,df,on='xy')

# dall.fillna(0)


tif_xy_pair = list(zip(df1.x, df1.y))

for index,row in df.iterrows():
    
    nearest_index = tif_xy_pair.index(min(tif_xy_pair, key=lambda x: distance(x, (row['x'],row['y']))))
    
    dall.loc[nearest_index,'value'] = row['value']
    
    # dall.loc[dall['x'] == row['x'] & (round(dall['y'],3) == round(row['y'],3))  , ['value']] = row['value']
    # dall.loc[(round(dall['x'],3) == round(row['x'],3)) & (round(dall['y'],3) == round(row['y'],3))  , ['value']] = row['value']
    # print((round(dall['x'],3) == round(row['x'],3)))
    # print(row['value'])


dall['value'] = dall['value'].fillna(0)
# dall['value'] = np.random.randint(0, 7, dall.shape[0])

In [17]:

dall.to_csv("map.xyz", index = False, header = None, sep = " ")
demn = gdal.Translate("map.tif", "map.xyz")
demn = None

In [18]:
df

,x,y,value
0,76.706308,12.284713,0
1,76.706583,12.284711,0
2,76.706859,12.284709,0
3,76.707135,12.284708,0
4,76.707411,12.284706,0
...,...,...,...
68,76.707121,12.282538,1
69,76.707397,12.282536,1
70,76.707672,12.282535,0
71,76.707948,12.282533,0


In [19]:
from IPython.display import Markdown
display (Markdown('Kharif Rabi Zaid <span style="color: #ffffff;background-color: #ffffff">color-indicator</span>'))
display (Markdown('Kharif Rabi ____ <span style="color: #00ffff;background-color: #00ffff">color-indicator</span>'))
display (Markdown('Kharif ____ Zaid <span style="color: #ff00ff;background-color: #ff00ff">color-indicator</span>'))
display (Markdown('______ Rabi Zaid <span style="color: #ffff00;background-color: #ffff00">color-indicator</span>'))
display (Markdown('Kharif ____ ____ <span style="color: #0000ff;background-color: #0000ff">color-indicator</span>'))
display (Markdown('______ Rabi ____ <span style="color: #00ff00;background-color: #00ff00">color-indicator</span>'))
display (Markdown('______ ____ Zaid <span style="color: #ff0000;background-color: #ff0000">color-indicator</span>'))

Kharif Rabi Zaid <span style="color: #ffffff;background-color: #ffffff">color-indicator</span>

Kharif Rabi ____ <span style="color: #00ffff;background-color: #00ffff">color-indicator</span>

Kharif ____ Zaid <span style="color: #ff00ff;background-color: #ff00ff">color-indicator</span>

______ Rabi Zaid <span style="color: #ffff00;background-color: #ffff00">color-indicator</span>

Kharif ____ ____ <span style="color: #0000ff;background-color: #0000ff">color-indicator</span>

______ Rabi ____ <span style="color: #00ff00;background-color: #00ff00">color-indicator</span>

______ ____ Zaid <span style="color: #ff0000;background-color: #ff0000">color-indicator</span>

In [20]:
mimg = gdal.Open('map.tif')
cimg = gdal.Open(tif_name)


In [21]:
pixel_predictions = (mimg.GetRasterBand(1)).ReadAsArray()
print(pixel_predictions)
op = (cimg.GetRasterBand(1)).ReadAsArray()
[cols, rows] = op.shape

print(np.unique(pixel_predictions))

[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 2 2 2 0 0 0 0]
 [0 0 0 2 3 2 2 0 0 0]
 [0 0 2 2 2 3 2 2 0 0]
 [0 0 2 3 3 3 2 2 2 0]
 [0 2 3 3 3 3 2 0 0 0]
 [0 0 3 3 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
[0 1 2 3]


In [22]:
dst_ds = driver.Create(final_name, rows, cols, 1, gdal.GDT_UInt16)
dst_ds.SetGeoTransform(cimg.GetGeoTransform())
dst_ds.SetProjection(cimg.GetProjection())
dst_ds.GetRasterBand(1).WriteArray((mimg.GetRasterBand(1)).ReadAsArray())
# dst_ds.GetRasterBand(2).WriteArray((mimg.GetRasterBand(2)).ReadAsArray())
# dst_ds.GetRasterBand(3).WriteArray((mimg.GetRasterBand(3)).ReadAsArray())
dst_ds.FlushCache()



band = dst_ds.GetRasterBand(1)

#making color
colors = gdal.ColorTable()

colors.SetColorEntry(0, (0, 0, 0))
colors.SetColorEntry(1, (255, 0, 0))
colors.SetColorEntry(2, (0, 255, 0))
colors.SetColorEntry(3, (255, 255, 0))
colors.SetColorEntry(4, (0, 0, 255))
colors.SetColorEntry(5, (255, 0, 255))
colors.SetColorEntry(6, (0,255, 255))
colors.SetColorEntry(7, (255,255, 255))

# colors.SetColorEntry(4, (194, 140, 124))
# colors.SetColorEntry(5, (214, 193, 156))

# # Change RBG for different colors
# colors.CreateColorRamp(0, (255, 0, 0), 126, (0, 0, 255))
# colors.CreateColorRamp(126, (0, 0, 255) , 252, (0, 0, 0))

band.SetRasterColorTable(colors)
band.SetRasterColorInterpretation(gdal.GCI_PaletteIndex)

del band, dst_ds
